# Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import math
import random
import timeit

from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

---

In [2]:
marburg = pd.read_pickle("../0_data/marburg_clean.pkl")
duisburg = pd.read_pickle('../0_data/duisburg_clean.pkl')
marburg_weather = pd.read_pickle("../0_data/weather/marburg_weather.pkl")
duisburg_weather = pd.read_pickle("../0_data/weather/duisburg_weather.pkl")

# Poly Tests

In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [4]:
def poly_ridge_model(degree, alpha):
    X_poly_train = PolynomialFeatures(degree = degree).fit_transform(X_train)
    lin_reg_Poly = Ridge(alpha = alpha, normalize = True, solver = 'lsqr')
    lin_reg_Poly.fit(X_poly_train, y_train)
    
    return lin_reg_Poly

In [5]:
def calc_errors(degree_range, alpha_range):
    degree_length = len(degree_range)
    alpha_length = len(alpha_range)

    errors_arr = np.empty((degree_length, alpha_length))
    for i, degree in enumerate(degree_range):
        for j, alpha in enumerate(alpha_range):
            degree = int(degree)
            X_poly_val =  PolynomialFeatures(degree = degree).fit_transform(X_val)
            errors_arr[i][j] = mean_squared_error(poly_ridge_model(degree, alpha).predict(X_poly_val), y_val)
    
    errors = pd.DataFrame(errors_arr, columns = alpha_range, index = degree_range)

    return errors.sort_index(ascending=False)

In [6]:
def split_data(X, y):
    # Do a 50-50 split first
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=34)

    # Now split X_test to achive 50-20-30 split
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=(0.2/0.5),random_state=34)

    for name, data in {'train': X_train, 'validation': X_val, 'test': X_test}.items():
        print("""{}
        {} entries
        ~{:.2f}% of total data

        """.format(name, len(data), len(data)/len(X)))
        
    return X_train, X_val, X_test, y_train, y_val, y_test 

In [7]:
def evaluate_model(model):
    mse = mean_squared_error(model.predict(X_test), y_test)
    mae = mean_absolute_error(model.predict(X_test), y_test)
    r2 = r2_score(model.predict(X_test), y_test)

    print("""Evaluation of model

    Mean-Squared-Error:             {:8.4f}
    Mean-Absolute-Error:            {:8.4f}
    
    Coefficient of determination:   {:8.4f} %
    """.format(mse, mae, r2*100))

## Marburg

We start by preparing the dataframe.  
First we resample the given data hourly.  
Then we extract some time related features from the datetime index and merge the resulting dataframe witht the weather data.

In [8]:
# peak demand
ma = pd.DataFrame(marburg.resample('H').count()["day"])
ma.rename(columns={'day': 'demand'}, inplace=True)

ma['dayofyear'] = ma.index.map(lambda datetime : datetime.dayofyear)
ma['week'] = ma.index.map(lambda datetime : datetime.week)
ma['weekday'] = ma.index.map(lambda datetime : datetime.weekday)
ma['is_weekday'] = ma['weekday'].map(lambda day : day < 5)
ma['hour'] = ma.index.map(lambda datetime : datetime.hour)

ma = ma.merge(marburg_weather, left_index=True, right_index=True)

In [9]:
ma

,demand,dayofyear,week,weekday,is_weekday,hour,temperature,precipitation
datetime,,,,,,,,
2019-01-20 00:00:00,7,20,3,6,False,0,-7.9,0.0
2019-01-20 01:00:00,8,20,3,6,False,1,-8.3,0.0
2019-01-20 02:00:00,5,20,3,6,False,2,-8.7,0.0
2019-01-20 03:00:00,1,20,3,6,False,3,-8.9,0.0
2019-01-20 04:00:00,2,20,3,6,False,4,-8.7,0.0
...,...,...,...,...,...,...,...,...
2020-01-20 19:00:00,36,20,4,0,True,19,-1.5,0.0
2020-01-20 20:00:00,31,20,4,0,True,20,-1.7,0.0
2020-01-20 21:00:00,16,20,4,0,True,21,-2.3,0.0


In [10]:
X = ma[['temperature', 'hour', 'precipitation', 'dayofyear','is_weekday']].values
y = ma['demand'].values

In [11]:
X_train, X_val, X_test, y_train, y_val, y_test  = split_data(X,y)

train
        4392 entries
        ~0.50% of total data

        
validation
        1757 entries
        ~0.20% of total data

        
test
        2635 entries
        ~0.30% of total data

        


In [ ]:
degree_range = np.linspace(2, 20, 10)
alpha_range = np.logspace(-5, 2, 10)

### LONG PROCESSING OPERATION ###
ma_errors_1 = calc_errors(degree_range, alpha_range)

In [ ]:
ma_vmin_1 = ma_errors_1.values.min()
ma_vmin_1

In [ ]:
ax = sns.heatmap(ma_errors_1,
                 xticklabels=ma_errors_1.columns.values.round(4),
                 yticklabels=ma_errors_1.index.values.round(4))
ax.set_xlabel('alpha')
ax.set_ylabel('degree') 
ax.set_title('MSE over sigma and alpha')

plt.show()

In [20]:
ma_errors_1

,0.000010,0.000017,0.000028,0.000046,0.000077,0.000129,0.000215,0.000359,0.000599,0.001000
10.000000,4639.644921,4638.360397,4636.218752,4632.649248,4626.703219,4616.807584,4600.364243,4573.110801,4528.132770,4454.425282
9.111111,3267.629849,3266.287194,3264.049191,3260.320646,3254.114021,3243.796689,3226.685739,3198.416741,3152.010274,3076.626398
8.222222,2139.801300,2138.156688,2135.417449,2130.859610,2123.288492,2152.135852,2132.719317,2100.878375,2049.243415,1967.026527
7.333333,1226.556590,1225.029313,1222.487734,1218.264950,1211.267412,1199.722677,1180.813842,1150.212279,1101.647313,1026.968757
6.444444,517.400918,516.081936,513.893199,510.273755,504.322754,494.630915,479.091584,454.802703,418.355231,226.108373
5.555556,97.149064,97.152545,97.158631,97.169545,97.189819,97.229152,97.309054,95.803967,96.187239,96.857162
4.666667,172.622132,172.533547,172.386269,172.141952,171.738145,171.074754,169.995660,168.268309,165.572865,161.528805
3.777778,220.609491,219.609329,217.970303,215.315272,211.094807,204.585811,182.929914,172.937709,160.039841,145.331589
2.888889,120.258342,120.259756,120.262122,120.266088,120.272758,120.284035,120.303250,120.336391,120.394521,120.498653
2.000000,120.258342,120.259756,120.262122,120.266088,120.272758,120.284035,120.303250,120.336391,120.394521,120.498653


In [ ]:
degree_range = np.linspace(2, 10, 10)
alpha_range = np.logspace(-5, -3, 10)

### LONG PROCESSING OPERATION ###
ma_errors_2 = calc_errors(degree_range, degree_range)